In [1]:
from mbientlab.warble import *
from mbientlab.metawear import *
from ctypes import cast, byref
from threading import Event
from time import sleep
import pandas as pd
from math import nan

In [2]:
e = Event()
address = None
def device_discover_task(result):
    global address
    if (result.has_service_uuid(MetaWear.GATT_SERVICE)):
        # grab the first discovered metawear device
        address = result.mac
        e.set()

BleScanner.set_handler(device_discover_task)
BleScanner.start()
e.wait()

BleScanner.stop()

In [3]:
# Initialize d and connect
d = MetaWear(address)
d.connect()
e = Event()

In [ ]:
freq = 100
acc_range = 16
gyro_range = 2000

libmetawear.mbl_mw_acc_set_odr(d.board, freq)
libmetawear.mbl_mw_acc_set_range(d.board, acc_range)
libmetawear.mbl_mw_acc_write_acceleration_config(d.board)

if freq == 800:
    gyro_freq = GyroBoschOdr._800Hz
elif freq == 400:
    gyro_freq = GyroBoschOdr._400Hz
elif freq == 200:
    gyro_freq = GyroBoschOdr._200Hz
elif freq == 100:
    gyro_freq = GyroBoschOdr._100Hz
elif freq == 50:
    gyro_freq = GyroBoschOdr._50Hz
elif freq == 25:
    gyro_freq = GyroBoschOdr._25Hz
else:
    gyro_freq = GyroBoschOdr._100Hz
    
if gyro_range == 2000:
    gyro_ra = GyroBoschRange._2000dps
elif gyro_range == 1000:
    gyro_ra = GyroBoschRange._1000dps
elif gyro_range == 500:
    gyro_ra = GyroBoschRange._500dps
elif gyro_range == 250:
    gyro_ra = GyroBoschRange._250dps
elif gyro_range == 125:
    gyro_ra = GyroBoschRange._125dps
else:
    gyro_ra = GyroBoschRange._2000dps
    
libmetawear.mbl_mw_gyro_bmi160_set_odr(d.board, gyro_freq)
libmetawear.mbl_mw_gyro_bmi160_set_range(d.board, gyro_ra)
libmetawear.mbl_mw_gyro_bmi160_write_config(d.board)

# Acc and gyro signals
acc = libmetawear.mbl_mw_acc_get_acceleration_data_signal(d.board)
gyro = libmetawear.mbl_mw_gyro_bmi160_get_rotation_data_signal(d.board)

# Create a logger
acc_logger = create_voidp(lambda fn: libmetawear.mbl_mw_datasignal_log(acc, None, fn), resource = "acc_logger")
gyro_logger = create_voidp(lambda fn: libmetawear.mbl_mw_datasignal_log(gyro, None, fn), resource = "gyro_logger")

# Start logger
libmetawear.mbl_mw_logging_start(d.board, 0)
# Turn on the accelerometer
libmetawear.mbl_mw_acc_enable_acceleration_sampling(d.board)
libmetawear.mbl_mw_gyro_bmi160_enable_rotation_sampling(d.board)
libmetawear.mbl_mw_acc_start(d.board)
libmetawear.mbl_mw_gyro_bmi160_start(d.board)


print("Logging data for 5s")
sleep(5.0)

# Turn off the accelerometer
libmetawear.mbl_mw_acc_stop(d.board)
libmetawear.mbl_mw_gyro_bmi160_stop(d.board)
libmetawear.mbl_mw_acc_disable_acceleration_sampling(d.board)
libmetawear.mbl_mw_gyro_bmi160_disable_rotation_sampling(d.board)
# Stop logging
libmetawear.mbl_mw_logging_stop(d.board)

print("Downloading data")
# Callback function to handle logger entries as we download them
def progress_update_handler(context, entries_left, total_entries):
    if (entries_left == 0):
        e.set()

# Function pointer and handlers for the logger download (LogDownloadHandler -> in the cbindings)
fn_wrapper = FnVoid_VoidP_UInt_UInt(progress_update_handler)
download_handler = LogDownloadHandler(context = None, received_progress_update = fn_wrapper, received_unknown_entry = cast(None, FnVoid_VoidP_UByte_Long_UByteP_UByte), received_unhandled_entry = cast(None, FnVoid_VoidP_DataP))

acc_data = {}
gyro_data = {}

def acc_data_handler(context, p):
    parsed = parse_value(p)
    acc_data[int(p.contents.epoch)] = {'x': parsed.x, 'y': parsed.y, 'z': parsed.z}
    print(f'Sensor: acc, epoch: {p.contents.epoch}, values: {parsed}')

def gyro_data_handler(context, p):
    parsed = parse_value(p)
    gyro_data[int(p.contents.epoch)] = {'x': parsed.x, 'y': parsed.y, 'z': parsed.z}
    print(f'Sensor: gyro, epoch: {p.contents.epoch}, values: {parse_value(p)}')
    
acc_callback = FnVoid_VoidP_DataP(acc_data_handler)
gyro_callback = FnVoid_VoidP_DataP(gyro_data_handler)

# Stop logger
libmetawear.mbl_mw_logger_subscribe(acc_logger, None, acc_callback)
libmetawear.mbl_mw_logger_subscribe(gyro_logger, None, gyro_callback)

# Download logger contents
libmetawear.mbl_mw_logging_download(d.board, 0, byref(download_handler))
e.wait()

libmetawear.mbl_mw_logging_clear_entries(d.board)


Exception ignored on calling ctypes callback function: <function create_voidp.<locals>.handler at 0x0000021118A0B6A0>
Traceback (most recent call last):
  File "C:\Users\jelte\AppData\Local\Programs\Python\Python311\Lib\site-packages\mbientlab\metawear\__init__.py", line 75, in handler
    result[0] = RuntimeError("Could not create " + (kwargs['resource'] if 'resource' in kwarg else "resource") ) if pointer == None else pointer
                                                                                        ^^^^^
NameError: name 'kwarg' is not defined


In [11]:
d.on_disconnect = lambda status: print ("we are disconnected!")
d.disconnect()

we are disconnected!


In [10]:
all_epochs = sorted(set(list(acc_data.keys()) + list(gyro_data.keys())))
data_df = pd.DataFrame(columns=['Timestamp', 'x_acc', 'y_acc', 'z_acc', 'x_gyro', 'y_gyro', 'z_gyro'])

for epoch in all_epochs:
    row = [float(epoch - all_epochs[0])]
    if epoch in acc_data.keys() and epoch in gyro_data.keys():
        row += [acc_data[epoch]['x'], acc_data[epoch]['y'], acc_data[epoch]['z'], gyro_data[epoch]['x'], gyro_data[epoch]['y'], gyro_data[epoch]['z']]
    elif epoch in acc_data.keys():
        row += [acc_data[epoch]['x'], acc_data[epoch]['y'], acc_data[epoch]['z'], nan, nan, nan]
    else:
        row += [nan, nan, nan, gyro_data[epoch]['x'], gyro_data[epoch]['y'], gyro_data[epoch]['z']]
    data_df.loc[len(data_df)] = row
    
data_df.head(100)

,Timestamp,x_acc,y_acc,z_acc,x_gyro,y_gyro,z_gyro
0,0.0,-0.649902,-0.076172,-0.798340,NaN,NaN,NaN
1,10.0,-0.656250,-0.075195,-0.807129,0.243902,0.609756,0.060976
2,20.0,-0.655273,-0.072754,-0.805176,-0.182927,0.426829,0.304878
3,29.0,-0.652344,-0.072266,-0.804688,NaN,NaN,NaN
4,31.0,NaN,NaN,NaN,-0.182927,0.548781,0.000000
...,...,...,...,...,...,...,...
95,905.0,-0.650879,-0.074219,-0.807129,-0.182927,0.487805,0.060976
96,915.0,-0.653320,-0.074707,-0.809082,-0.060976,0.426829,0.060976
97,926.0,-0.653320,-0.074707,-0.809082,-0.121951,0.548781,0.060976
98,934.0,-0.651367,-0.074219,-0.807617,-0.121951,0.609756,0.060976


In [ ]:
# d.connect()
# # Stops data logging
# libmetawear.mbl_mw_logging_stop(d.board)
# # Clear the logger of saved entries
# libmetawear.mbl_mw_logging_clear_entries(d.board)
# Remove all macros on the flash memory
libmetawear.mbl_mw_macro_erase_all(d.board)
# Restarts the board after performing garbage collection
libmetawear.mbl_mw_debug_reset_after_gc(d.board)
print("Erase logger, state, and macros")
libmetawear.mbl_mw_debug_disconnect(d.board)
d.disconnect()
print("Disconnected")

In [ ]:
libmetawear.mbl_mw_macro_erase_all(d.board)
print(1)
libmetawear.mbl_mw_debug_disconnect(d.board)
print(2)
d.disconnect()
print(3)